In [14]:
import pandas as pd
import numpy as np
import re
import requests
from io import StringIO
import os

In [15]:
url = "http://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.CD?format=json&per_page=20000"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()[1]  # Get the data from the second element of the response
    df = pd.json_normalize(data)  # Convert the JSON data into a pandas DataFrame
    print(df.head())
else:
    print("Request failed with status code:", response.status_code)

  countryiso3code  date        value unit obs_status  decimal    indicator.id  \
0             AFE  2022          NaN                        1  NY.GDP.PCAP.CD   
1             AFE  2021  1549.772730                        1  NY.GDP.PCAP.CD   
2             AFE  2020  1363.540741                        1  NY.GDP.PCAP.CD   
3             AFE  2019  1512.270553                        1  NY.GDP.PCAP.CD   
4             AFE  2018  1564.734340                        1  NY.GDP.PCAP.CD   

                indicator.value country.id                country.value  
0  GDP per capita (current US$)         ZH  Africa Eastern and Southern  
1  GDP per capita (current US$)         ZH  Africa Eastern and Southern  
2  GDP per capita (current US$)         ZH  Africa Eastern and Southern  
3  GDP per capita (current US$)         ZH  Africa Eastern and Southern  
4  GDP per capita (current US$)         ZH  Africa Eastern and Southern  


In [16]:
#cambiamos el nombre de columnas
df = df.rename(columns={'countryiso3code': 'country_code', 'indicator.id': 'indicator_id', 'indicator.value': 'indicator_value', 'country.id': 'country_id', 'country.value': 'country', 'date': 'year'})


In [17]:
df.columns

Index(['country_code', 'year', 'value', 'unit', 'obs_status', 'decimal',
       'indicator_id', 'indicator_value', 'country_id', 'country'],
      dtype='object')

In [18]:
format_currency = lambda x: "${:,.2f}".format(x)
df['gdp'] = df['value'].apply(format_currency)
df = df.drop('value', axis=1)

In [19]:
df

,country_code,year,unit,obs_status,decimal,indicator_id,indicator_value,country_id,country,gdp
0,AFE,2022,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern,$nan
1,AFE,2021,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern,"$1,549.77"
2,AFE,2020,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern,"$1,363.54"
3,AFE,2019,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern,"$1,512.27"
4,AFE,2018,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern,"$1,564.73"
...,...,...,...,...,...,...,...,...,...,...
16753,ZWE,1964,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe,$282.38
16754,ZWE,1963,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe,$277.53
16755,ZWE,1962,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe,$275.97
16756,ZWE,1961,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe,$279.33


In [20]:
df['gdp'] = df['gdp'].str.replace('[$,]', '')  # Step 1
df['gdp'] = pd.to_numeric(df['gdp'], errors='coerce')  # Step 2

/var/folders/5x/gn6s3css5x508rs__lb9ck2w0000gn/T/ipykernel_5508/903355698.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['gdp'] = df['gdp'].str.replace('[$,]', '')  # Step 1


In [21]:
df

,country_code,year,unit,obs_status,decimal,indicator_id,indicator_value,country_id,country,gdp
0,AFE,2022,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern,NaN
1,AFE,2021,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern,1549.77
2,AFE,2020,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern,1363.54
3,AFE,2019,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern,1512.27
4,AFE,2018,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZH,Africa Eastern and Southern,1564.73
...,...,...,...,...,...,...,...,...,...,...
16753,ZWE,1964,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe,282.38
16754,ZWE,1963,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe,277.53
16755,ZWE,1962,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe,275.97
16756,ZWE,1961,,,1,NY.GDP.PCAP.CD,GDP per capita (current US$),ZW,Zimbabwe,279.33


In [22]:
df.to_csv('../Users/zakaria/Desktop/ironhack/MiniProject Tableau/PIB.csv', index = False)

In [23]:
print(df['gdp'].dtypes)


float64
